In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
load_dotenv()

True

In [9]:
def get_chatbot_response(client, model_name, messages, temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000
    ).choices[0].message.content

    return response

In [31]:
client = OpenAI(
    api_key=os.getenv("OPENAI_TOKEN"),
    base_url=os.getenv("OPENAI_CHATBOT_URL")
)
model_name = os.getenv("MODEL_NAME")

# Get LLM Response

In [10]:
messages = [{"role": "user", "content": "What is the capital of France?"}]
response = get_chatbot_response(client, model_name, messages)

In [11]:
response

'The capital of France is Paris.'

# Prompt Engineering

## Structured output

In [12]:
system_prompt = """
You are a helpful assistant that answer questions about capital of countries

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
[
{
    "country": the country that you will get the capital of
    "capital": the capital of the country stated
}
]
"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": "What is the capital of France?"})
response = get_chatbot_response(client, model_name, messages)
print(response)

[
    {
        "country": "France",
        "capital": "Paris"
    }
]


In [14]:
type(response)

str

In [15]:
json_response = json.loads(response)
json_response

[{'country': 'France', 'capital': 'Paris'}]

In [18]:
type(json_response[0]),json_response[0]['capital']

(dict, 'Paris')

## Input structuring

In [20]:
user_input = """
Get me the capital of the following countries:
```
1. Italy
2. Germany
3. France
```
"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": user_input})
response = get_chatbot_response(client, model_name, messages)
print(response)

[
    {
        "country": "Italy",
        "capital": "Rome"
    },
    {
        "country": "Germany",
        "capital": "Berlin"
    },
    {
        "country": "France",
        "capital": "Paris"
    }
]


## Chain of thought

In [23]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{
    "result": the final number resulted from calculating the equation above
}
"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

{
    "result": 300000.0
}


In [22]:
259/2*8654+91072*33-12971

4113098.0

In [24]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{
    "steps": this where you solve the equation bit by bit following the PEMDAS order of operations. You need to show your work and calculate each step leading to the final result. Feel free to write in free text
    "result": the final number resulted from calculating the equation above
}
"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

{
    "steps": [
        "Start with the equation: 259/2*8654+91072*33-12971",
        "First, perform the division: 259/2 = 129.5",
        "Now substitute back into the equation: 129.5*8654+91072*33-12971",
        "Next, perform the multiplication: 129.5*8654 = 1111963",
        "Now substitute back into the equation: 1111963 + 91072*33 - 12971",
        "Next, perform the multiplication: 91072*33 = 3000366",
        "Now substitute back into the equation: 1111963 + 3000366 - 12971",
        "Now perform the addition: 1111963 + 3000366 = 4112329",
        "Now substitute back into the equation: 4112329 - 12971",
        "Finally, perform the subtraction: 4112329 - 12971 = 4109358"
    ],
    "result": 4109358
}


# Retrieval-Augmented Generation

In [27]:
user_prompt = """
What's new in iphone 16?
"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

As of my last update in October 2023, Apple had not officially released the iPhone 16, so I can't provide specific details about its features or specifications. However, Apple typically announces new iPhone models in September each year, so you may want to check the latest news or Apple's official website for the most current information on the iPhone 16, including any new features, design changes, camera improvements, and performance enhancements. If you have any other questions or need information on previous iPhone models, feel free to ask!


In [28]:
iphone_16 = """
Iphone 16 is the latest smartphone from Apple, featuring a sleek design, improved camera capabilities, and enhanced performance. It comes with the new A17 chip, which provides faster processing speeds and better energy efficiency. The camera system has been upgraded with advanced computational photography features, allowing for stunning low-light photos and improved video recording capabilities. Additionally, the iPhone 16 supports 5G connectivity for faster internet speeds and has a longer-lasting battery life compared to its predecessors.
"""

In [30]:
user_prompt = f"""
{iphone_16}

What's new in iphone 16?
"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

The iPhone 16 introduces several notable features and improvements:

1. **A17 Chip**: The new A17 chip enhances processing speeds and energy efficiency, providing a smoother user experience and better performance for demanding applications.

2. **Camera Upgrades**: The camera system has been significantly improved with advanced computational photography features, enabling stunning low-light photography and enhanced video recording capabilities.

3. **5G Connectivity**: The iPhone 16 supports 5G, allowing for faster internet speeds and improved connectivity.

4. **Battery Life**: The device boasts a longer-lasting battery compared to previous models, ensuring extended usage without frequent recharging.

5. **Sleek Design**: The iPhone 16 features a modern and sleek design, continuing Apple's trend of aesthetic innovation.

These enhancements make the iPhone 16 a compelling choice for users looking for cutting-edge technology and improved performance in a smartphone.


In [ ]:
motorcycle -- 2
car        -- 3
banana     -- 45

car-motorcycle = 1
banana-car = 42

In [32]:
output = client.embeddings.create(
    input=user_prompt,
    model="text-embedding-3-small"
)